In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as ss
import scipy.io
import os
import pandas as pd
from sklearn import neighbors, datasets
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#Ustawiam domyślne wielkości wykresów
plt.rcParams['figure.figsize'] = [20, 10]
plt.rcParams['figure.dpi'] = 100 
plt.rcParams.update({"axes.grid" : True, "grid.color": "black", "grid.linestyle": '--', "grid.linewidth": 0.5 })
#plt.subplots_adjust(top = 1, bottom = 0.3, wspace=1, hspace=2)

In [ ]:
file = '/content/drive/Shareddrives/BR4IN_IO_Hackaton_2021/P1_high1.mat'
data = scipy.io.loadmat(file)
channel_names = ['Fz', 'C3', 'Cz', 'CP1', 'CPz', 'CP2', 'Pz']

Fs = 256
T = len(data['y'][:,0])/Fs 
dt = 1/Fs
t = np.arange(0,T,dt)

#plt.title(channel_names[0])
#plt.plot(t, data['y'][:,0])

In [ ]:
def slice_signal(sig, tags, Fs):
    '''
    sig = single channel signal as a list or ndarray
    tags = tag file as a list or ndarray containing stimuli names (classes). Starting point (as a sample) of the stimuli corresponds to the index of the class in this array. tags and sig have the same length.
    Fs = integer, sampling frequency

    This function slices the signal based on the tag fule and returns a dictionary containing all signal slices for a particular stimuli class.
    '''
    result = {'1':[],'-1':[],'2':[]} #dict keys made specifically for this case
    ind = 0
    for item in tags:
        if item != [0]:
          result[str(item[0])].append(sig[ind-int(0.1*Fs): ind+int(0.6*Fs)])
        ind += 1
    return result#len(result['2']) powinno byc 60

In [ ]:
#print(len(slice_signal(data['y'], data['trig'], 256)['1'][0][0]))

In [ ]:
def preprocess_signal(sig, Fs, num_channels):
    for i in range(num_channels):
        proc_sig = preprocess_signal(sig[:,i],Fs = Fs)
      
        [b_h, a_h] = ss.butter(2, 0.1, fs = Fs, btype = "highpass") 
        [b_l, a_l] = ss.butter(2, 30, fs = Fs, btype = "lowpass") 
        #[b_z, a_z] = ss.iirnotch(50, 3, fs = Fs)
        [b_z,a_z] = ss.cheby2(2, rs = 25, Wn = [49,51], btype='bandstop', analog=0, output='ba', fs=Fs)

        print(b_z, b_l, b_h)

        sig_f = ss.filtfilt(b_h, a_h, proc_sig)
        sig_f = ss.filtfilt(b_l, a_l, sig_f)
        sig_f = ss.filtfilt(b_z, a_z, sig_f)

    # sig = sig.T

    # [b_h, a_h] = ss.butter(2, 0.1, fs = Fs, btype = "highpass") 
    # [b_l, a_l] = ss.butter(2, 30, fs = Fs, btype = "lowpass") 
    # [b_z, a_z] = ss.iirnotch(50, 3, fs = Fs)
    # #[b_z,a_z] = ss.cheby2(2, rs = 25, Wn = [49,51], btype='bandstop', analog=0, output='ba', fs=Fs)

    # print(b_z, b_l, b_h)

    # sig_f = ss.filtfilt(b_h, a_h, sig)
    # sig_f = ss.filtfilt(b_l, a_l, sig_f)
    # sig_f = ss.filtfilt(b_z, a_z, sig_f)

    # sig_f = sig_f.T

    return sig_f

In [ ]:
#os.listdir('/content/drive/Shareddrives/BR4IN_IO_Hackaton_2021')
path = '/content/drive/Shareddrives/BR4IN_IO_Hackaton_2021/'

num_channels = 8
channel_names = ['Fz', 'C3', 'Cz', 'CP1', 'CPz', 'CP2', 'Pz']

P1 = {'1':[],'-1':[],'2':[]}
P2 = {'1':[],'-1':[],'2':[]}
for file in os.listdir(path):
    data = scipy.io.loadmat(path+file)
    Fs = data['fs']
    tags = data['trig']
    ''' 
    for i in range(num_channels):
        print(np.shape(data['y'][:,i]))
        psig = preprocess_signal(data['y'][:,i],Fs = Fs)
        slice_signal(psig, tags = data['trig'], Fs = Fs) #przygotowane dane z jednego kanału z jednego pliku
    ''' 
    psig = data['y']
    #psig = preprocess_signal(data['y'], Fs[0][0], num_channels=num_channels)
    if file[:2] == 'P1':
        res = slice_signal(psig, tags = data['trig'], Fs = Fs)#przygotowane dane z każdego kanału z jednego pliku
        P1 =  {x: P1.get(x, 0) + res.get(x, 0) for x in set(P1).union(res)}
    elif file[:2] == 'P2':
        res = slice_signal(psig, tags = data['trig'], Fs = Fs)#przygotowane dane z każdego kanału z jednego pliku
        P2 =  {x: P2.get(x, 0) + res.get(x, 0) for x in set(P2).union(res)}

print(P1.keys(), len(P1['1']), len(P1['-1']), len(P1['2']))
print(P2.keys(), len(P2['1']), len(P2['-1']), len(P2['2']))
print(len(P1['1']), len(P1['1'][0]),len(P1['1'][0][0]))
print(len(P2['1']), len(P2['1'][0]),len(P2['1'][0][0]))

r = 1920
patients = np.array([np.vstack((P1['1'],P1['-1'],P1['2'])), np.vstack((P2['1'],P2['-1'],P2['2']))]) 
labels = np.zeros(r)
labels[:240] = -1
labels[240:1680] = 1
labels[1680:] = 2
print(patients.shape)
print(labels)
len(labels)

dict_keys(['2', '1', '-1']) 240 1440 240
dict_keys(['2', '1', '-1']) 240 1440 240
240 178 8
240 178 8
(2, 1920, 178, 8)
[-1. -1. -1. ...  2.  2.  2.]


1920

In [ ]:
df = pd.read_csv('hackathon_knn.csv').drop(['Unnamed: 0'],axis=1)
#print([c for c in df.columns])

channel_names = ['Fz', 'C3', 'Cz', 'C4', 'CP1', 'CPz', 'CP2', 'Pz']

scores = []
i = 0
for pat in [1,2]:
  for ch in channel_names:
    i += 1
    data = df[(df['channel'] == ch) & (df['patient'] == pat)]

    X_train = data[['amplitude', 'mean', 'std', 'med', 'skewness', 'kurtosis', 'slope_change', 'power', 'energy']] 
    y_train = data['class']

    #Split dataset into training set and test set
    Xtr, Xte, ytr, yte = train_test_split(X_train, y_train, test_size=0.3) #70% training and 30% test

    #Create a Gaussian Classifier
    n_neighbors = 20
    #clf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance', metric = 'euclidean')
    clf = RandomForestClassifier(n_estimators=300)

    clf.fit(Xtr, ytr)
    ypr = clf.predict(Xte)
    #Model Accuracy, how often is the classifier correct?
    print(i, 'Accuracy for patient', pat, 'on channel', ch, ': ',metrics.accuracy_score(yte, ypr))
    scores.append(metrics.accuracy_score(yte, ypr))
print('Mean accuracy score: ', np.mean(scores))

1 Accuracy for patient 1 on channel Fz :  0.734375
2 Accuracy for patient 1 on channel C3 :  0.71875
3 Accuracy for patient 1 on channel Cz :  0.7239583333333334
4 Accuracy for patient 1 on channel C4 :  0.71875
5 Accuracy for patient 1 on channel CP1 :  0.7361111111111112
6 Accuracy for patient 1 on channel CPz :  0.7291666666666666
7 Accuracy for patient 1 on channel CP2 :  0.7361111111111112
8 Accuracy for patient 1 on channel Pz :  0.7361111111111112
9 Accuracy for patient 2 on channel Fz :  0.7534722222222222
10 Accuracy for patient 2 on channel C3 :  0.7430555555555556
11 Accuracy for patient 2 on channel Cz :  0.7309027777777778
12 Accuracy for patient 2 on channel C4 :  0.7604166666666666
13 Accuracy for patient 2 on channel CP1 :  0.7274305555555556
14 Accuracy for patient 2 on channel CPz :  0.7517361111111112
15 Accuracy for patient 2 on channel CP2 :  0.7447916666666666
16 Accuracy for patient 2 on channel Pz :  0.7378472222222222
Mean accuracy score:  0.7364366319444444


In [ ]:
scores = []
i = 0
for pat in [1,2]:
  for ch in channel_names:
    i += 1
    data = df[(df['channel'] == ch) & (df['patient'] == pat)]

    X_train = data[['amplitude', 'mean', 'std', 'med', 'skewness', 'kurtosis', 'slope_change', 'power', 'energy']] 
    y_train = data['class']

    #Split dataset into training set and test set
    Xtr, Xte, ytr, yte = train_test_split(X_train, y_train, test_size=0.3) #70% training and 30% test

    #Create a Gaussian Classifier
    n_neighbors = 20
    clf3 = neighbors.KNeighborsClassifier(n_neighbors, weights='distance', metric = 'euclidean')
    #clf = RandomForestClassifier(n_estimators=300)

    clf3.fit(Xtr, ytr)
    ypr = clf3.predict(Xte)
    #Model Accuracy, how often is the classifier correct?
    print(i, 'Accuracy for patient', pat, 'on channel', ch, ': ',metrics.accuracy_score(yte, ypr))
    scores.append(metrics.accuracy_score(yte, ypr))
print('Mean accuracy score: ', np.mean(scores))

1 Accuracy for patient 1 on channel Fz :  0.6944444444444444
2 Accuracy for patient 1 on channel C3 :  0.7222222222222222
3 Accuracy for patient 1 on channel Cz :  0.6927083333333334
4 Accuracy for patient 1 on channel C4 :  0.6892361111111112
5 Accuracy for patient 1 on channel CP1 :  0.7065972222222222
6 Accuracy for patient 1 on channel CPz :  0.7065972222222222
7 Accuracy for patient 1 on channel CP2 :  0.7065972222222222
8 Accuracy for patient 1 on channel Pz :  0.6996527777777778
9 Accuracy for patient 2 on channel Fz :  0.7638888888888888
10 Accuracy for patient 2 on channel C3 :  0.7378472222222222
11 Accuracy for patient 2 on channel Cz :  0.7065972222222222
12 Accuracy for patient 2 on channel C4 :  0.71875
13 Accuracy for patient 2 on channel CP1 :  0.6840277777777778
14 Accuracy for patient 2 on channel CPz :  0.7239583333333334
15 Accuracy for patient 2 on channel CP2 :  0.7274305555555556
16 Accuracy for patient 2 on channel Pz :  0.7256944444444444
Mean accuracy score:  

In [ ]:
df2 = pd.read_csv('hackathon_knn(1).csv').drop(['Unnamed: 0'],axis=1)

scores = []
i = 0
for pat in [1,2]:
  if pat == 1:
    file_names = ['P1_high1', 'P1_high2', 'P1_low1', 'P1_low2']
  elif pat == 2:
    file_names = ['P2_high1', 'P2_high2', 'P2_low1', 'P2_low2']
  for ch in channel_names:
    for fname in file_names:
      i += 1
      data = df2[(df2['channel'] == ch) & (df2['patient'] == pat) & (df2['file_name'] == fname)]
      
      X_train = data[['amplitude', 'mean', 'std', 'med', 'skewness', 'kurtosis', 'slope_change', 'power', 'energy']] 
      y_train = data['class']

      #Split dataset into training set and test set
      Xtr, Xte, ytr, yte = train_test_split(X_train, y_train, test_size=0.3) #70% training and 30% test

      #Create a Gaussian Classifier
      n_neighbors = 10
      #clf = neighbors.KNeighborsClassifier(n_neighbors, weights='distance', metric = 'euclidean')
      clf2 = RandomForestClassifier(n_estimators=300)

      clf2.fit(Xtr, ytr)
      ypr = clf2.predict(Xte)
      #Model Accuracy, how often is the classifier correct?
      print(i, 'Accuracy for patient', pat, 'on channel', ch, 'in file', fname, ': ',metrics.accuracy_score(yte, ypr))
      scores.append(metrics.accuracy_score(yte, ypr))
print('Mean accuracy score: ', np.mean(scores))

1 Accuracy for patient 1 on channel Fz in file P1_high1 :  0.625
2 Accuracy for patient 1 on channel Fz in file P1_high2 :  1.0
3 Accuracy for patient 1 on channel Fz in file P1_low1 :  1.0
4 Accuracy for patient 1 on channel Fz in file P1_low2 :  0.5972222222222222
5 Accuracy for patient 1 on channel C3 in file P1_high1 :  0.6180555555555556
6 Accuracy for patient 1 on channel C3 in file P1_high2 :  1.0
7 Accuracy for patient 1 on channel C3 in file P1_low1 :  1.0
8 Accuracy for patient 1 on channel C3 in file P1_low2 :  0.6458333333333334
9 Accuracy for patient 1 on channel Cz in file P1_high1 :  0.5972222222222222
10 Accuracy for patient 1 on channel Cz in file P1_high2 :  1.0
11 Accuracy for patient 1 on channel Cz in file P1_low1 :  1.0
12 Accuracy for patient 1 on channel Cz in file P1_low2 :  0.5833333333333334
13 Accuracy for patient 1 on channel C4 in file P1_high1 :  0.6805555555555556
14 Accuracy for patient 1 on channel C4 in file P1_high2 :  1.0
15 Accuracy for patient 1 o

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten, LSTM
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

def DeepConvNet(nb_classes, Chans = 8, Samples = 178,
                dropoutRate = 0.5):
    """ Keras implementation of the Deep Convolutional Network as described in
    Schirrmeister et. al. (2017), Human Brain Mapping.
    
    This implementation assumes the input is a 2-second EEG signal sampled at 
    128Hz, as opposed to signals sampled at 250Hz as described in the original
    paper. We also perform temporal convolutions of length (1, 5) as opposed
    to (1, 10) due to this sampling rate difference. 
    
    Note that we use the max_norm constraint on all convolutional layers, as 
    well as the classification layer. We also change the defaults for the
    BatchNormalization layer. We used this based on a personal communication 
    with the original authors.
    
                      ours        original paper
    pool_size        1, 2        1, 3
    strides          1, 2        1, 3
    conv filters     1, 5        1, 10
    
    Note that this implementation has not been verified by the original 
    authors. 
    
    """

    # start the model
    input_main   = Input((1, Chans, Samples))
    block1       = Conv2D(25, (1, 5), 
                                 input_shape=(1, Chans, Samples),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)), padding = 'same')(input_main)
    block1       = Conv2D(25, (Chans, 1),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)), padding = 'same')(block1)
    block1       = BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1)(block1)
    block1       = Activation('tanh')(block1)
    block1       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block1)
    block1       = Dropout(dropoutRate)(block1)
  
    block2       = Conv2D(50, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)), padding = 'same')(block1)
    block2       = BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1)(block2)
    block2       = Activation('tanh')(block2)
    block2       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block2)
    block2       = Dropout(dropoutRate)(block2)
    
    block3       = Conv2D(100, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)), padding = 'same')(block2)
    block3       = BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1)(block3)
    block3       = Activation('tanh')(block3)
    block3       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2))(block3)
    block3       = Dropout(dropoutRate)(block3)
    
    block4       = Conv2D(200, (1, 5),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)), padding = 'same')(block3)
    block4       = BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1)(block4)
    block4       = Activation('tanh')(block4)
    block4       = MaxPooling2D(pool_size=(1, 2), strides=(1, 2), padding = 'same')(block4)
    block4       = Dropout(dropoutRate)(block4)
    
    flatten      = Flatten()(block4)
    
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)



In [ ]:
# need these for ShallowConvNet
def square(x):
    return K.square(x)

def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))   


def ShallowConvNet(nb_classes, Chans = 64, Samples = 128, dropoutRate = 0.5):
    """ Keras implementation of the Shallow Convolutional Network as described
    in Schirrmeister et. al. (2017), Human Brain Mapping.
    
    Assumes the input is a 2-second EEG signal sampled at 128Hz. Note that in 
    the original paper, they do temporal convolutions of length 25 for EEG
    data sampled at 250Hz. We instead use length 13 since the sampling rate is 
    roughly half of the 250Hz which the paper used. The pool_size and stride
    in later layers is also approximately half of what is used in the paper.
    
    Note that we use the max_norm constraint on all convolutional layers, as 
    well as the classification layer. We also change the defaults for the
    BatchNormalization layer. We used this based on a personal communication 
    with the original authors.
    
                     ours        original paper
    pool_size        1, 35       1, 75
    strides          1, 7        1, 15
    conv filters     1, 13       1, 25    
    
    Note that this implementation has not been verified by the original 
    authors. We do note that this implementation reproduces the results in the
    original paper with minor deviations. 
    """

    # start the model
    input_main   = Input((1, Chans, Samples))
    block1       = Conv2D(40, (1, 13), 
                                 input_shape=(1, Chans, Samples),
                                 kernel_constraint = max_norm(2., axis=(0,1,2)), padding = 'same')(input_main)
    block1       = Conv2D(40, (Chans, 1), use_bias=False, 
                          kernel_constraint = max_norm(2., axis=(0,1,2)), padding = 'same')(block1)
    block1       = BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1)(block1)
    block1       = Activation(square)(block1)
    block1       = AveragePooling2D(pool_size=(1, 35), strides=(1, 7), padding = 'same')(block1)
    block1       = Activation(log)(block1)
    block1       = Dropout(dropoutRate)(block1)
    flatten      = Flatten()(block1)
    dense        = Dense(nb_classes, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_main, outputs=softmax)

In [ ]:
def light(nb_classes, Chans = 8, Samples = 178, 
             dropoutRate = 0.5, kernLength = 64, F1 = 4, 
             D = 2, F2 = 8, norm_rate = 0.25, dropoutType = 'Dropout'):
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (1, Chans, Samples))

    ##################################################################

    uno       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (1, Chans, Samples),
                                   use_bias = False)(input1)
    # layer = tf.keras.layers.Dropout(.2, input_shape=(2,))
    
    dos = Dense(1000, name = 'dense2', 
                         kernel_constraint = max_norm(norm_rate))(uno)
    dos = Dense(500, name = 'dense21', 
                         kernel_constraint = max_norm(norm_rate))(dos)                      

    flatten      = Flatten(name = 'flatten')(dos)

    tres = Dense(nb_classes, name = 'dense3', 
                         kernel_constraint = max_norm(norm_rate))(flatten)

    softmax      = Activation('softmax', name = 'softmax')(tres)
    
    return Model(inputs=input1, outputs=softmax)

In [ ]:

def EEGNet(nb_classes, Chans = 8, Samples = 178, 
             dropoutRate = 0.5, kernLength = 64, F1 = 4, 
             D = 2, F2 = 8, norm_rate = 0.25, dropoutType = 'Dropout'):
    
    if dropoutType == 'SpatialDropout2D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout2D '
                         'or Dropout, passed as a string.')
    
    input1   = Input(shape = (1, Chans, Samples))

    ##################################################################
    block1       = Conv2D(F1, (1, kernLength), padding = 'same',
                                   input_shape = (1, Chans, Samples),
                                   use_bias = False)(input1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = DepthwiseConv2D((Chans, 1), use_bias = False, 
                                   depth_multiplier = D,
                                   depthwise_constraint = max_norm(1.), padding = 'same')(block1)
    block1       = BatchNormalization(axis = 1)(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling2D((1, 4), padding = 'same')(block1)
    block1       = dropoutType(dropoutRate)(block1)
    
    block2       = SeparableConv2D(F2, (1, 16),
                                   use_bias = False, padding = 'same')(block1)
    block2       = BatchNormalization(axis = 1)(block2)
    block2       = Activation('elu')(block2)
    block2       = AveragePooling2D((1, 8), padding = 'same')(block2)
    block2       = dropoutType(dropoutRate)(block2)
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(nb_classes, name = 'dense', 
                         kernel_constraint = max_norm(norm_rate))(flatten)
    softmax      = Activation('softmax', name = 'softmax')(dense)
    
    return Model(inputs=input1, outputs=softmax)


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8
#
# Gibran Fuentes-Pineda <gibranfp@unam.mx>
# IIMAS, UNAM
# 2018
#
"""
Functions to read P300 Speller database as NumPy files. 
"""
import numpy as np
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

class roc_callback(tf.keras.callbacks.Callback):
    """
    Class for Keras callback to calculate ROC curve and AUC
    """
    
    def __init__(self,training_data, validation_data):
        self.x = training_data[0]
        self.y = training_data[1]
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]
        
        
    def on_train_begin(self, logs={}):
        return
        
    def on_train_end(self, logs={}):
        return
        
    def on_epoch_begin(self, epoch, logs={}):
        return
        
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.x)
        roc = roc_auc_score(self.y, y_pred)
        y_pred_val = self.model.predict(self.x_val)
        roc_val = roc_auc_score(self.y_val, y_pred_val)
        
        print('AUC Train: {0} AUC Valid: {1}'.format(roc, roc_val))
        
        return
        
    def on_batch_begin(self, batch, logs={}):
        return
        
    def on_batch_end(self, batch, logs={}):
        return
    
def load_db(datafile, labelsfile):
    """
    Function that reads P300 Speller database from UAM
    """
    print('Reading P300 Speller database')
    data = np.load(datafile)
    labels = np.load(labelsfile)
    print('Data shape = {0}, Labels shape = {1}'.format(data.shape, labels.shape))
    
    return data, labels

class EEGChannelScaler():
    """
    Class to scale each channel of an EEG signal separately
    """
    def __init__(self, n_channels = 8):
        """
        Defines and initialize a standard scaler for each channel
        """
        self.n_channels_ = n_channels
        self.sc_ = []
        for c in range(self.n_channels_):
            self.sc_.append(StandardScaler())
            
    def fit_transform(self, X):
        """
        Fits the standard scaler of scikit-learn for each channel using the training data
        """
        if (X.shape[2] != self.n_channels_):
            print('Error: Expected {0} channels, got {1} instead.'.format(self.n_channels_, X.shape[2]))

        for c in range(self.n_channels_):
            X[:, :, c] = self.sc_[c].fit_transform(X[:, :, c])

        self.fitted_ = True
        
        return X
    
    def transform(self, X):
        """
        Scales an array (nrows, ncols, nchannels) for each channel separately
        """
        for c in range(self.n_channels_):
            X[:, :, c] = self.sc_[c].transform(X[:, :, c])

        return X

def make_trial_average(X, y, n_trials = 2, pos_samples = 10000, neg_samples = 10000):
    """
    Function that makes trial averages
    """
    print('Generating {0} positive and {1} negative samples of {2}-trial averages'.format(pos_samples, neg_samples, n_trials))

    X_pos = X[y == 1, :, :]
    X_neg = X[y == 0, :, :]
    
    X_avg = np.zeros((pos_samples + neg_samples, X.shape[1], X.shape[2]))
    y_avg = np.zeros(pos_samples + neg_samples)
    y_avg[:pos_samples] = 1
    for i in range(pos_samples):
        pos_trials = np.random.choice(X_pos.shape[0], n_trials)
        X_avg[i, :, :] = np.mean(X_pos[pos_trials, :, :], axis = 0)

    for i in range(neg_samples):
        neg_trials = np.random.choice(X_neg.shape[0], n_trials)
        X_avg[pos_samples + i, :, :] = np.mean(X_neg[neg_trials, :, :], axis = 0)
            
    perm = np.random.permutation(pos_samples + neg_samples)
    X_avg = X_avg[perm, :, :]
    y_avg = y_avg[perm]
        
    return X_avg, y_avg
        
def stack_trials(X, y, n_trials = 2, pos_samples = 1000, neg_samples = 1000):
    """
    Function that makes trial averages
    """
    print('X shape = {0}, y shape = {1}'.format(X.shape, y.shape))
    print('Generating {0} positive and {1} negative samples of {2}-trial averages'.format(pos_samples, neg_samples, n_trials))
    
    X_pos = X[y == 1, :, :]
    X_neg = X[y == 0, :, :]
    
    X_stack = np.zeros((pos_samples + neg_samples, X.shape[1], X.shape[2], n_trials))
    y_stack = np.zeros(pos_samples + neg_samples)
    for i in range(pos_samples):
        pos_trials = np.random.choice(X_pos.shape[0], n_trials)
        X_stack[i, :, :, :] = X_pos[pos_trials, :, :].transpose(1, 2, 0)
        y_stack[i] = 1
        
        for i in range(neg_samples):
            neg_trials = np.random.choice(X_neg.shape[0], n_trials)
            X_stack[pos_samples + i, :, :, :] = X_neg[neg_trials, :, :].transpose(1, 2, 0)
            y_stack[pos_samples + i] = 0
            
            perm = np.random.permutation(pos_samples + neg_samples)
            X_stack = X_stack[perm, :, :]
            y_stack = y_stack[perm]
            
    return X_stack, y_stack
        
def balance_data(X, y, n_samples = 1000, btype = 'downsample'):
    """
    Function to balance data
    """
    X_pos = X[y == 1]
    X_neg = X[y == 0]

    if btype == 'downsample':
        X_neg = resample(X_neg, replace = False, n_samples = n_samples)
    else:
        X_pos = resample(X_pos, replace = True, n_samples = n_samples)

    X_balanced = np.concatenate((X_neg, X_pos))
    y_balanced = np.zeros(X_neg.shape[0] + X_pos.shape[0])
    y_balanced[X_neg.shape[0]:] = 1
    perm = np.random.permutation(X_balanced.shape[0])
    
    return X_balanced[perm], y_balanced[perm]

In [ ]:
labels = np.append(labels, labels)

In [ ]:
import argparse
import sys
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
import tensorflow
from sklearn.model_selection import *
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam

def evaluate_cross_subject_model(data, labels, modelpath):
    """
    Trains and evaluates DeepConvNet for each subject in the P300 Speller database
    using random cross validation.
    """
    n_sub = data.shape[0]
    n_ex_sub = data.shape[1]
    n_samples = data.shape[2]
    n_channels = data.shape[3]

    aucs = np.zeros(n_sub)
    data = data.reshape((n_sub * n_ex_sub, n_samples, n_channels))

    data = data[:1920]
    labels = labels[:1920]

    # labels = labels.reshape((n_sub * n_ex_sub))

    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.33)

    # groups = np.array([i for i in range(n_sub) for j in range(n_ex_sub)])

    # cv = LeaveOneGroupOut()
    # for k, (t, v) in enumerate(cv.split(data, labels, groups)):
    # X_train, y_train, X_test, y_test = data[t], labels[t], data[v], labels[v]

    # rg = np.random.choice(t, 1)
    # sv = groups[t] == groups[rg]
    # st = np.logical_not(sv)
    # X_train, y_train, X_valid, y_valid = data[t][st], labels[t][st], data[t][sv], labels[t][sv]
    # print("Partition {0}: train = {1}, valid = {2}, test = {3}".format(k, X_train.shape, X_valid.shape, X_test.shape))
    # print("Groups train = {0}, valid = {1}, test = {2}".format(np.unique(groups[t][st]),
    #                                                            np.unique(groups[t][sv]),
    #                                                            np.unique(groups[v])))
        
    # channel-wise feature standarization
    sc = EEGChannelScaler(n_channels = n_channels)
    X_train = np.swapaxes(sc.fit_transform(X_train)[:, np.newaxis, :], 2, 3)
    X_test = np.swapaxes(sc.transform(X_test)[:, np.newaxis, :], 2, 3)



    model = EEGNet(3, dropoutRate = 0.5, Chans = n_channels, Samples = n_samples)
    # model = light(3, dropoutRate = 0.5, Chans = n_channels, Samples = n_samples)
    # model = DeepConvNet(3, dropoutRate = 0, Chans = n_channels, Samples = n_samples)
    # model = ShallowConvNet(3, dropoutRate = 0.25, Chans = n_channels, Samples = n_samples)
    print(model.summary())
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy')

    es = EarlyStopping(monitor = 'val_loss', patience = 50, mode = 'min', restore_best_weights = True)
    model.fit(X_train,
              to_categorical(y_train),
              batch_size = 256,
              epochs = 200,
              validation_split = 0.2,
              callbacks = [es])
    
    yhat_probs = model.predict(X_test)
    yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_probs = yhat_probs
    print(yhat_classes)
    print(y_test)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes, average='micro')
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes, average='micro')
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes, average='micro')
    print('F1 score: %f' % f1)

    # aucs[k] = roc_auc_score(y_test, proba_test[:, 1])
    # print('P{0} -- AUC: {1}'.format(k, aucs[k]))
    K.clear_session()
    
    # np.savetxt(modelpath + '/aucs.npy', aucs)



np.random.seed(1)
tensorflow.random.set_seed(2)

evaluate_cross_subject_model(patients, labels, '/content/drive')
        

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1, 8, 178)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 1, 8, 4)           45568     
_________________________________________________________________
batch_normalization (BatchNo (None, 1, 8, 4)           4         
_________________________________________________________________
depthwise_conv2d (DepthwiseC (None, 1, 8, 8)           64        
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 8, 8)           4         
_________________________________________________________________
activation (Activation)      (None, 1, 8, 8)           0         
_________________________________________________________________
average_pooling2d (AveragePo (None, 1, 2, 8)           0     

In [ ]:
#1920/2  elu 1000 drugi pacjent drop zmieniony na 0.7 z 0.5

In [ ]:
#for i, j in zip()